In [137]:
# t-test  F-test  R^2
from sklearn.datasets import fetch_california_housing as fetch_data
import numpy as np
import pandas as pd
import scipy.stats as st


In [138]:
data = fetch_data()
housing_data = pd.DataFrame(data.data)
housing_data.columns = data.feature_names
targets = data.target
display(housing_data.head())
display(targets)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [225]:
class OLS_model(object):
    
    def __init__(self,y,X):
        data_x = X.copy()
        data_x.insert(0,'intercept',np.ones(len(data_x)))
        self.columns =data_x.columns
        self.X = np.mat(data_x)
        self.b = None
        self.y = y.copy()
        self.N = self.X.shape[0]
        self.K = self.X.shape[1]
        
    def fit_data(self,output=True):
        
        self.b = np.array(np.dot(np.dot(np.linalg.inv(np.dot(self.X.T,self.X)),self.X.T), np.mat(self.y).T))
        
        for i in range(self.K):
            print(f'vairable {self.columns[i]} coef is {self.b[i][0]}')
        return self.b
                          
    def predict_y(self,X_pred):
        y_hat =np.dot(X_pred,self.b) 
        return y_hat
                          
    def t_test(self,alpha=0.05):
        X_XT_inv = np.linalg.inv(np.dot(self.X.T,self.X))
        y_hat =np.array(np.dot(self.X,self.b).T)[0]
        sigma_hat = np.sum((y_hat-self.y)**2)/(self.N-self.K)
        #print(X_XT_inv)
        for i in range(self.K-1):
            sk = X_XT_inv[i,i]
            #print( f'sk value is{sk}')
            Tk = self.b[i][0]/np.sqrt(sigma_hat*sk)
            #print(f'Tk value is{Tk}')
            if abs(Tk)>abs(st.t.ppf(alpha/2,df = self.N-self.K)):
                print(f'reject H0. T of {i} is {Tk}')
            else:
                print(f'can\'t reject H0. T of {i} is {Tk}\n' )
        return
    
    def R_square(self):
        y_mean = np.mean(self.y)
        y_hat = np.array(np.dot(self.X,self.b).T)[0]
        TSS = np.sum(np.square(self.y-y_mean))
        RSS = np.sum(np.square(self.y-y_hat))
        self.R_2 = 1-RSS/TSS
        return self.R_2
    
    def f_test(self,alpha=0.05):
        r2 = self.R_square()
        F = r2/(self.K-1)/((1-r2)/(self.N-self.K))
        if abs(F)>abs(st.f.ppf(1-alpha,dfn=self.K-1,dfd=self.N-self.K)):
            print(f'F of function is {F}, reject H0')
        else:
            print(f'F of function is {F}, can\'t reject H0')
        return
                          

In [226]:
ols_m = OLS_model(targets,housing_data)

In [227]:
ols_m.fit_data()

vairable intercept coef is -36.94192020630866
vairable MedInc coef is 0.43669329313679733
vairable HouseAge coef is 0.009435778033390466
vairable AveRooms coef is -0.10732204139378053
vairable AveBedrms coef is 0.6450656935308543
vairable Population coef is -3.9763894208274435e-06
vairable AveOccup coef is -0.003786542654993447
vairable Latitude coef is -0.4213143775180024
vairable Longitude coef is -0.43451375466458014


array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

In [228]:
ols_m.t_test()

reject H0. T of 0 is -56.06653255400927
reject H0. T of 1 is 104.05380188420229
reject H0. T of 2 is 21.143205211533065
reject H0. T of 3 is -18.23536534862611
reject H0. T of 4 is 22.927563478648217
can't reject H0. T of 5 is -0.8372759090301649

reject H0. T of 6 is -7.768623843923927
reject H0. T of 7 is -58.54136565050525


In [229]:
for i in range(11):
    print(np.dot(ols_m.X.T,ols_m.X)[i][i])

[[ 2.06400000e+04  7.98906495e+04  5.91119000e+05  1.12054555e+05
   2.26353751e+04  2.94218400e+07  6.33783225e+04  7.35441620e+05
  -2.46791870e+06]]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [230]:
np.dot(ols_m.X.T,ols_m.X)

matrix([[ 2.06400000e+04,  7.98906495e+04,  5.91119000e+05,
          1.12054555e+05,  2.26353751e+04,  2.94218400e+07,
          6.33783225e+04,  7.35441620e+05, -2.46791870e+06],
        [ 7.98906495e+04,  3.83723229e+05,  2.22928568e+06,
          4.65439543e+05,  8.64612451e+04,  1.14096929e+08,
          2.52959028e+05,  2.83996841e+06, -9.55369356e+06],
        [ 5.91119000e+05,  2.22928568e+06,  2.01984850e+07,
          3.11067738e+06,  6.38694833e+05,  7.55482945e+08,
          1.85071039e+06,  2.10688691e+07, -7.07362324e+07],
        [ 1.12054555e+05,  4.65439543e+05,  3.11067738e+06,
          7.34686462e+05,  1.43399883e+05,  1.55555196e+08,
          3.41507443e+05,  4.00431634e+06, -1.34011476e+07],
        [ 2.26353751e+04,  8.64612451e+04,  6.38694833e+05,
          1.43399883e+05,  2.94589973e+04,  3.15329544e+07,
          6.88775051e+04,  8.07997152e+05, -2.70624360e+06],
        [ 2.94218400e+07,  1.14096929e+08,  7.55482945e+08,
          1.55555196e+08,  3.153295

In [231]:
np.dot(ols_m.X.T,ols_m.X)[0,0]

20640.0

In [232]:
ols_m.columns

Index(['intercept', 'MedInc', 'HouseAge', 'AveRooms', 'AveBedrms',
       'Population', 'AveOccup', 'Latitude', 'Longitude'],
      dtype='object')

In [233]:
ols_m.b

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

In [234]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [235]:
exo = sm.add_constant(housing_data)
ls = sm.OLS(targets,exo)

In [236]:
result=ls.fit()

In [237]:
result.params

const        -36.941920
MedInc         0.436693
HouseAge       0.009436
AveRooms      -0.107322
AveBedrms      0.645066
Population    -0.000004
AveOccup      -0.003787
Latitude      -0.421314
Longitude     -0.434514
dtype: float64

In [238]:
ols_m.b

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

In [239]:
np.dot(np.dot(np.linalg.inv(np.dot(ols_m.X.T,ols_m.X)),ols_m.X.T),ols_m.y.T)

matrix([[-3.69419202e+01,  4.36693293e-01,  9.43577803e-03,
         -1.07322041e-01,  6.45065694e-01, -3.97638942e-06,
         -3.78654265e-03, -4.21314378e-01, -4.34513755e-01]])

In [240]:
ols_m.f_test(0.05)

F of function is 3970.360812801199, reject H0


In [241]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     3970.
Date:                Sun, 10 Oct 2021   Prob (F-statistic):               0.00
Time:                        14:47:17   Log-Likelihood:                -22624.
No. Observations:               20640   AIC:                         4.527e+04
Df Residuals:                   20631   BIC:                         4.534e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -36.9419      0.659    -56.067      0.000     -38.233     -35.650
MedInc         0.4367      0.004    104.054      0.000       0.428       0.445
HouseAge       0.0094      0.000     21.143      0.000       0.009       0.010
AveRooms      -0.1073      0.006    -18.235      0.000      -0.119      -0.096
AveBedrms      0.6451      0.028     22.928      0.000       0.590       0.700
Population -3.976e-06   4.75e-06     -0.837      0.402   -1.33e-05    5.33e-06
AveOccup      -0.0038      0.000     -7.769      0.000      -0.005      -0.003
Latitude      -0.4213      0.007    -58.541      0.000      -0.435      -0.407
Longitude     -0.4345      0.008    -57.682      0.000      -0.449      -0.420
==============================================================================
Omnibus:                     4393.650   Durbin-Watson:                   0.885
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14087.596
Skew:                           1.082   Prob(JB):                         0.00
Kurtosis:                       6.420   Cond. No.                     2.38e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.38e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""